# Deal with training, dev, test datasets

In [1]:
import re
import os
import pathlib
import random
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
import warnings
import shutil
import pickle

from PDBPreprocess import Pdb
from preprocessing import getContent
warnings.filterwarnings("ignore")

def getSeq(file):
    for record in SeqIO.parse(file, "pdb-atom"):
        seq = str(record.seq)
        return seq

In [3]:
# 把pdbdatabase中所有的cyclic copy过来

pdb_path ="D:/Work_Study/Work/Syneron/Code/pdbdb/pdbfiles_v1.0/pdbfiles_clean.tar/pdbfiles_clean/pdbfiles/"
cyclic_path = "cyclic_data"
bond_file = "cyclic_position_bond.txt"

with open(bond_file,"r") as f:
    lines = f.readlines()
    for line in lines:
        pdbid = line.split(": ")[0]+".pdb"
        shutil.copyfile(os.path.join(pdb_path,pdbid), os.path.join(cyclic_path,re.sub("_",".",pdbid)))

# 然后把cys_data（pdbdatabase和apptest的cys）合并在一起变成了cyclic_data

cyclic_data 共 1818 条
过筛一遍5-40长度的


In [ ]:
pdbfiles = os.listdir(cyclic_path)
pdbid_5_40 = []
for file in tqdm(pdbfiles):
    length = len(getSeq(os.path.join(cyclic_path,file)))
    if length >=5 and length <=40:
        pdbid_5_40.append(pathlib.Path(file).stem)

In [5]:
print(len(pdbid_5_40))

1597


In [29]:
def filter(path, pdbid_5_40,bond):
    dirs = os.listdir(path)
    for file in tqdm(dirs):
        # 1a0m.A.pdb -> 1a0m.A
        if file[:-4] not in pdbid_5_40:
            os.remove(os.path.join(path, file))
    dirs = os.listdir(path)
    print(len(dirs))

    output = []
    with open(bond,"r") as f:
        lines = f.readlines()
        for line in lines:
            line = re.sub("_",".",line)
            if line.split(": ")[0] in pdbid_5_40:
                output.append(line)
    with open(bond,"w") as f:
        f.writelines(output)
filter(cyclic_path,pdbid_5_40,"cyclic_position_bond.txt")

100%|██████████| 1597/1597 [00:00<00:00, 64057.53it/s]


1597


5-40长度还剩下1597条二硫键环肽。现在再随机找1597条5-40之间的线性肽

In [7]:
# 筛选5-40之间，标准的
aa_codes = {
    'A': 'ALA', 'C': 'CYS', 'D': 'ASP', 'E': 'GLU',
    'F': 'PHE', 'G': 'GLY', 'H': 'HIS', 'K': 'LYS',
    'I': 'ILE', 'L': 'LEU', 'M': 'MET', 'N': 'ASN',
    'P': 'PRO', 'Q': 'GLN', 'R': 'ARG', 'S': 'SER',
    'T': 'THR', 'V': 'VAL', 'Y': 'TYR', 'W': 'TRP'}


def containNonStandard(seq):
    for aa in seq:
        try:
            if not aa_codes[aa]:
                return True
        except KeyError:
            return True
    return False

# D:/Work_Study/Work/Syneron/Code/pdbdb/pdbfiles_v1.0/pdbfiles_clean.tar/pdbfiles_clean
count = 0
allpdbpath = "D:/Work_Study/Work/Syneron/Code/pdbdb/pdbfiles_v1.0/pdbfiles_clean.tar/pdbfiles_clean/pdbfiles"
allpdbfiles = os.listdir(allpdbpath)
allpdblists = []
for file in tqdm(allpdbfiles):
    try :
        seq = getSeq(os.path.join(allpdbpath,file))
        length = len(seq)
        if length >=5 and length <=40 and not containNonStandard(seq):
                allpdblists.append(file)
    except Exception:
        print(file)
        continue
# 这些print出来的是unk】
print(len(allpdblists))

1ati_C.pdb
1ati_D.pdb
1bcc_I.pdb
1efg_B.pdb
1efg_C.pdb
1fe1_E.pdb
1fe1_F.pdb
1fip_C.pdb
1fka_N.pdb
1izl_X.pdb
1qzv_J.pdb
1qzv_K.pdb
1r42_B.pdb
1r42_C.pdb
1r42_D.pdb
1r42_E.pdb
1rjl_D.pdb
1rqf_O.pdb
1rqf_P.pdb
1seb_C.pdb
1z56_E.pdb
1z56_F.pdb
2a79_D.pdb
2chn_C.pdb
2hpe_S.pdb
2j9n_B.pdb
2kvj_A.pdb
2o01_K.pdb
3a0b_N.pdb
3a0h_N.pdb
3gt8_X.pdb
3hm6_C.pdb
3j5q_A.pdb
3jc2_3.pdb
3jd5_s.pdb
3mmv_X.pdb
4fbx_B.pdb
4uq8_a.pdb
4uq8_d.pdb
4uq8_g.pdb
4uq8_m.pdb
4uq8_u.pdb
5dfz_G.pdb
7c0i_D.pdb
7jps_F.pdb
7jps_G.pdb
7lgj_E.pdb
7od6_F.pdb
7od7_E.pdb
7od8_E.pdb
7oew_E.pdb
7pzn_M.pdb
7txu_E.pdb
8853


In [8]:
with open('all_pdb_5_40.pkl', 'wb') as f:
    pickle.dump(allpdblists, f)

In [9]:
import pickle
with open('all_pdb_5_40.pkl', 'rb') as f:
    allpdblists = pickle.load(f)
# 重命名一下linear，然后随机选取1128条5-40长度之间不和cys重复的，还不能是非标准的

linearlist = [re.sub("_",".",a) for a in allpdblists]
duplicate = []
for i in pdbid_5_40:
    if i in linearlist:
        duplicate.append(i)
for i in duplicate:
    linearlist.remove(i)


linearlist = random.choices(linearlist,k=1123)
assert len(set(linearlist).intersection(pdbid_5_40)) == 0
print(len(linearlist))

NameError: name 'pdbid_5_40' is not defined

In [ ]:
# TODO: 看一下10fold怎么做resample
#
# 用的是pdb database中所有的二硫键成环的环肽（标准氨基酸）1128个 5-40之间的
# 然后数据增强:可以用69个一环头尾相连的环肽，对成环位置做序列位置循环（按照氨基酸的长度全部循环了，就比如原本长度10，那么就再创10-1个新的，需要结构文件也循环）。因为本身对一环的也比较重视，所以增加了712个
# 那么总共就是1597+712 = 2309 的环肽
# 再学一点肽的特性，因为数据还是太少了，增加了2309条线性肽
# 然后划分数据集，然后开始搭建模型


# 实际上是1597+1597条，没有做data augmentation

1597条环肽，1597条线性肽。在环肽中，有部分是来自apptest且不在pdbdatabase中，所以还需要更改一下cyclic_position_bond.txt。
把apptest的补进去。
bond的是来自pdbdb中的，cys部分是来自detectcyclic
所以现在把所有的位点信息都合并到cyclic_position_bond里面

In [30]:
cys = open("cys_bond.txt","r").readlines()
bond = set([line.split(": ")[0] for line in open("cyclic_position_bond.txt","r").readlines()])
more = []
for c in cys:
    if c.split(": ")[0] not in bond:
        more.append(c)
with open("cyclic_position_bond.txt","a") as f:
    f.writelines(more)

目前所有的位点信息都在cyclic_position_bond，所有环肽与线性肽都在cyclic_data。
后面需要更新distance和torsion。
然后再划分数据集

In [2]:
# 创建一个pdb类，去处理distance和torsion
pdb = Pdb(download_path="linear_data/")
pdb.getDistance("ca_distance_linear/","CA")
pdb.getDistance("cb_distance_linear/","CB")
pdb.getTorsion()

Start calculating CA distance...
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bbr.F.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bde.A.pdb
non-standard: 1bd

100%|██████████| 1281/1281 [01:48<00:00, 11.84it/s]


In [4]:
# 所有有计算distance问题的都去除了
f = open("non_standard_linear.txt","r")
non_standard = [line.strip() for line in f.readlines()]
f.close()

for file in os.listdir("linear_data"):
    if file in non_standard:
        os.remove(os.path.join("linear_data",file))
for file in os.listdir("ca_distance_linear"):
    if pathlib.Path(file).stem+".pdb" in non_standard:
        os.remove(os.path.join("ca_distance_linear",file))
for file in os.listdir("cb_distance_linear"):
    if pathlib.Path(file).stem+".pdb" in non_standard:
        os.remove(os.path.join("cb_distance_linear",file))
res = []
with open("torsion_linear.txt","r") as f:
    for line in f.readlines():
        if not line.split(": ")[0]+".pdb" in non_standard:
            res.append(line)
with open("torsion_linear.txt","w") as f:
    f.writelines(res)

In [5]:
# 再把所有torsion有问题的一起去掉。出现三个None及以上就有问题
torsion_removed = []
rest = []
with open("torsion_linear.txt","r") as f:
    for line in f.readlines():
        if line.count("None") > 2:
            torsion_removed.append(line.split(": ")[0])
        else:
            rest.append(line)
with open("torsion_linear.txt","w") as f:
    f.writelines(rest)

In [6]:
rest = []
with open("torsion_linear.txt","r") as f:
    for line in f.readlines():
        rest.append(line.split(": ")[0])

In [7]:
for file in os.listdir("linear_data"):
    if not pathlib.Path(file).stem in rest:
        os.remove(os.path.join("linear_data",file))
for file in os.listdir("ca_distance_linear"):
    if not pathlib.Path(file).stem in rest:
        os.remove(os.path.join("ca_distance_linear",file))
for file in os.listdir("cb_distance_linear"):
    if not pathlib.Path(file).stem in rest:
        os.remove(os.path.join("cb_distance_linear",file))

最后剩下1123环肽，883线性肽。有效distance和torsion的
分别在data中，以及ca_cb_distance 和torsion中(移除了test，val与train结构小于4rmsd相似的，避免泄露)

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
X_id = pd.DataFrame(os.listdir("data"))
# 在0.8*2006处切分，再0.9*2006处切分
X_id_train, X_id_validation, X_id_test = np.split(X_id.sample(frac=1), [int(.8*len(X_id)),
int(.9*len(X_id))])

In [2]:
from preprocessing import getContent

# seq, cyclic_matrix, ca, cb, phi, psi
data = {}
cyclic_position_file = "cyclic_position_bond.txt"
ca_distance_path = "ca_distance"
cb_distance_path = "cb_distance"
torsion_file = "torsion.txt"



index = 0
for id in X_id_validation[0]:
    pdb_file = os.path.join("data",id)
    seq, cyclic_matrix, ca, cb, phi, psi = getContent(pdb_file, cyclic_position_file, ca_distance_path, cb_distance_path, torsion_file)
    data[index] = {"seq": seq,
                "cyclic_mat": cyclic_matrix,
                 "ca_dis": ca,
                 "cb_dis": cb,
                 "phi": phi,
                 "psi":psi}
    index += 1

with open("val_data_info.pkl","wb") as f:
   pickle.dump(data,f)

data1 = {}
index = 0
for id in X_id_test[0]:
    pdb_file = os.path.join("data",id)
    seq, cyclic_matrix, ca, cb, phi, psi = getContent(pdb_file, cyclic_position_file, ca_distance_path, cb_distance_path, torsion_file)
    data1[index] = {"seq": seq,
                "cyclic_mat": cyclic_matrix,
                 "ca_dis": ca,
                 "cb_dis": cb,
                 "phi": phi,
                 "psi":psi}
    index += 1

with open("test_data_info.pkl","wb") as f:
   pickle.dump(data1,f)


data2 = {}
index = 0
for id in X_id_train[0]:
    pdb_file = os.path.join("data",id)
    seq, cyclic_matrix, ca, cb, phi, psi = getContent(pdb_file, cyclic_position_file, ca_distance_path, cb_distance_path, torsion_file)
    data2[index] = {"seq": seq,
                "cyclic_mat": cyclic_matrix,
                 "ca_dis": ca,
                 "cb_dis": cb,
                 "phi": phi,
                 "psi":psi}
    index += 1

with open("train_data_info.pkl","wb") as f:
   pickle.dump(data2,f)

In [1]:
import pickle
# with open("test_data_info.pkl","rb") as f:
#     test_data = pickle.load(f)
# with open("val_data_info.pkl","rb") as f:
#     val_data = pickle.load(f)
with open("test_data_info.pkl","rb") as f:
    train_data = pickle.load(f)

new_train_data = {}
index = 0
for key,value in train_data.items():
    new_train_data[index] = {"seq":key,
                             "cyclic_mat": value["cyclic_mat"],
                 "ca_dis": value["ca_dis"],
                 "cb_dis": value["cb_dis"],
                 "phi": value["phi"],
                 "psi":value["psi"]}
    index += 1
with open("test_data_info.pkl","wb") as f:
   pickle.dump(new_train_data,f)

In [2]:
import pickle

with open("tr_data_info.pkl","rb") as f:
    train_data = pickle.load(f)
print(train_data[0])

{'seq': 'PKLEPWKHP', 'cyclic_mat': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'ca_dis': [[0.0, 3.8200457, 5.7138925, 9.420793, 11.180257, 12.988958, 10.171886, 10.400562, 8.601403, 10000, 10000, 10000], [3.8200457, 0.0, 3.7731044, 6.992218, 9.407675, 11.8179245, 9.817878, 11.33534, 10.151163, 10000, 10000, 10000], [5.7138925, 3.7731044, 0.0, 3.8065743, 6.495268, 8.259331, 6.257869, 8.484455, 8.402638, 10000, 10000, 10000], [9.420793, 6.992218, 3.8065743, 0.0, 3.855066, 5.7492766, 5.6199327, 8.946539, 9.726846, 10000, 10000, 10000], [11.180257, 9.407675, 6.495268, 3.855066, 0.0, 3.8523495, 5.439492, 8.068916, 8.6098385, 10000, 10000, 10000], [12.988958, 11.8179245, 8.259331, 5.7492766, 3.8523495, 0.0, 3.8112676, 6.603933, 8.7943735, 10000, 10000, 10000], [10.171886, 9.81

In [ ]:
remove = []
path = "../../ca_distance"
files = os.listdir(path)
for file in files:
    with open(os.path.join(path,file),"rb") as f:
        content = pickle.load(f)
        for i in range(len(content)):
            if 10000 in content[i]:
                remove.append(pathlib.Path(file).stem)
                break
print(len(remove))
print(len(os.listdir("../../data")))
for pdb_chain in remove:
    try:
        os.remove(os.path.join("../../data",pdb_chain+".pdb"))
    except:
        continue

print(len(os.listdir("../../data")))

In [6]:
import pickle
with open("train_data_info.pkl","rb") as f:
   a = pickle.load(f)
print(len(a))

892
